# Generative AI Application Evaluation and Governance

## Overview

- Setup an environment to evaluate two assistants that perform summarization

- Implement a simple benchmark to compare two assistant models (🤖 vs 👾)

- Evaluate performance using MLflow

- Explore MLFlow resources on Databricks

## Setup env

In [0]:
# %run ./setup_env/env_benchmark_eval

In [0]:
import warnings
warnings.filterwarnings('ignore')
from rich import print
import json
import tqdm
import pandas as pd
from langchain_community.chat_models import ChatDatabricks
from langchain_core.messages import SystemMessage, HumanMessage
import mlflow

In [0]:
JSON_WRITER_SUMMARIES_PATH = '/Volumes/studies/databricks-dev/data_json/writer_summaries.json'

SERVING_MODELS = {
    'gpt-5-1': 'databricks-gpt-5-1',  # disabled
    'gpt-oss-20b': 'databricks-gpt-oss-20b',  # disabled
    'meta-llama-8b': 'databricks-meta-llama-3-1-8b-instruct',  # enabled
    'qwen-80b': 'databricks-qwen3-next-80b-a3b-instruct',  # enabled
    'llama-maverick-400b': 'databricks-llama-4-maverick', # enabled
    'gemma-12b': 'databricks-gemma-3-12b'  # enabled  
}

## Utils    

In [0]:
def load_json(json_name):
    with open(json_name, 'r', encoding='utf-8') as f:
        data = json.load(f)
        return data


def run_llm_summarization(llm_model: ChatDatabricks, input_text: str) -> str:
    '''Run a LLM model for summarize the input text. Returns a one sentence summary.'''

    system_prompt='''
    You are a helpful assistant that summarizes text. Given a text input, you need to return a summary in only one sentence. You are specialized in summarizing news. Do not change the sentiment of each news item (e.g., positive remains positive, negative remains negative, and so on). Be concise, consistent, and preserve the core facts.
    '''
    
    res = llm_model.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=input_text)
    ])
    
    print(res.content)
    
    return res

### Load dataset

In [0]:
df_writer_summaries = pd.DataFrame(load_json(JSON_WRITER_SUMMARIES_PATH))
df_writer_summaries.rename(columns={'summary': 'human_summary'}, inplace=True)
print(df_writer_summaries.shape)
df_writer_summaries.head()

(302, 3)

,article_id,article,human_summary
0,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Nick Schofield is riding Spring Heeled in the ...
1,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Preparation is taking place for a horse ridin...
2,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Nick Scholfield will travel to Ireland and is ...
3,b3168ab4857d4190ac3b2eb46d096f81,Dr Mehmet Oz's fellow faculty members at Colum...,Faculty at Columbia University have written an...
4,b3168ab4857d4190ac3b2eb46d096f81,Dr Mehmet Oz's fellow faculty members at Colum...,"Celebrity doctor, Mehmet Oz, is being attacked..."


In [0]:
one_sample = df_writer_summaries.iloc[0].to_dict()
print(f'**Article:**\n{one_sample['article']}')
print(f'**Summary:**\n{one_sample['human_summary']}')

**Article:**
Nick Scholfield is lined up to ride Spring Heeled in the Grand National at Aintree on April 11.

Nick Scholfield has been lined up to ride Jim Culloty’s Spring Heeled in the Crabbie’s Grand National at Aintree on
Saturday week.

Scholfield had been expected to partner Paul Nicholls-trained Sam Winner, who was pulled up in the Cheltenham Gold 
Cup, in the £1million race.

But the champion trainer  said on Wednesday it was unfair to tie Scholfield down to a gelding which is far from 
certain to run when the mount on another leading definite contender is being offered.

Scholfield, who has ridden in six Nationals and finished third in 2013 on Teaforthree, will travel to Ireland to 
sit on Spring Heeled at Culloty’s County Cork stable on Friday.

Nicholls said: ‘I have not made up my mind if I am going to run Sam Winner yet and Nick needed a decision.

‘I did not want to get into a situation next week when I had to say "sorry mate, he is not running" and did not 
want to stop him getting a good ride.

‘I have not pressed any buttons on any of the horses who ran at Cheltenham. That will happen over the weekend and 
early next week. I don’t want to run unless I am really happy.

‘I have plenty of other lads who could ride Sam Winner if he runs and would not be afraid to use Will Biddick or 
Harry Skelton.’

Spring Heeled (right) wins the Fulke Walwyn Kim Muir Challenge Cup at Cheltenham last year.

Spring Heeled, winner of Fulke Walwyn Kim Muir Challenge Cup at last season’s Cheltenham Festival, has been given a
National preparation.

The eight-year-old has run only once since finishing fourth to Road To Riches in the Galway Plate in July when he 
was fourth of five in the Bobbyjo Chase at Fairyhouse in February.

Racemail revealed on Wednesday that Culloty would have two runners in the National.

Robbie McNamara will ride his 2014 Gold Cup winner Lord Windermere.

Scholfield rides Teaforthree (front) as the horse jumps the last fence at Aintree in the 2013 Grand National.

McNamara said: ‘It's a great ride to get and I'm looking forward to it. I've ridden him before in a Grade One in 
Leopardstown and I was supposed to ride him in the Hennessy there as well, but I broke my collarbone the day 
before. I'm delighted to get back on him.’

With Nigel Twiston-Davies-trained Double Ross another confirmed non runner, David Pipe’s well supported Soll 
appears guaranteed a run at the bottom of the weights.

Luke Morris became the first jockey to ride 100 winners during an All Weather Flat racing season when a double at 
Chelmsford on Wednesday aboard Giantouch and Middle East Pearl carried him to 101 successes for the campaign.

**Summary:**
Nick Schofield is riding Spring Heeled in the Crabbie's Grand National on Saturday. Schofield was expected to ride 
Sam Winner. Says Schofield, "I have plenty of other lads who could ride Sam Winner..." Spring Heeled has only run 
once since finishing fourth in the Galway Plate.

## Define models to compare the summarization task

In [0]:
N_MAX_SAMPLES_EVAL = 2  #  limit for testing

In [0]:
df_writer_summaries.iloc[0]

article_id                        0adb86356834452298d180104ff54179
article          Nick Scholfield is lined up to ride Spring Hee...
human_summary    Nick Schofield is riding Spring Heeled in the ...
Name: 0, dtype: object

In [0]:
df_writer_summaries.iloc[0].copy().article

'Nick Scholfield is lined up to ride Spring Heeled in the Grand National at Aintree on April 11.\n\nNick Scholfield has been lined up to ride Jim Culloty’s Spring Heeled in the Crabbie’s Grand National at Aintree on Saturday week.\n\nScholfield had been expected to partner Paul Nicholls-trained Sam Winner, who was pulled up in the Cheltenham Gold Cup, in the £1million race.\n\nBut the champion trainer  said on Wednesday it was unfair to tie Scholfield down to a gelding which is far from certain to run when the mount on another leading definite contender is being offered.\n\nScholfield, who has ridden in six Nationals and finished third in 2013 on Teaforthree, will travel to Ireland to sit on Spring Heeled at Culloty’s County Cork stable on Friday.\n\nNicholls said: ‘I have not made up my mind if I am going to run Sam Winner yet and Nick needed a decision.\n\n‘I did not want to get into a situation next week when I had to say "sorry mate, he is not running" and did not want to stop him 

### ✍️ `Human` (ground truth)

In [0]:
'''Run a iteration of summarization task over a dataframe `df` according to model_name in {human, llm_a, llm_b}. Returns a list of summaries.
'''

def run_human_summary(df: pd.DataFrame = None) -> str:
    res = []
    for _, df_i in df.iterrows():
        res.append(df_i['human_summary'])
    return res

In [0]:
print(run_human_summary(df=df_writer_summaries.head(2)))

[
    'Nick Schofield is riding Spring Heeled in the Crabbie\'s Grand National on Saturday. Schofield was expected to
ride Sam Winner. Says Schofield, "I have plenty of other lads who could ride Sam Winner..." Spring Heeled has only 
run once since finishing fourth in the Galway Plate.  ',
    ' Preparation is taking place for a horse riding tournament, The Grand National. A jockey, Nick Scholfield, is 
deciding whether to ride one of two horses for this competition: Spring Heeled or Sam Winner.  '
]

### 🤖 `meta-llama-3-1-8b-instruct`

In [0]:
endpoint_llm_a = SERVING_MODELS['meta-llama-8b']
llm_model_a = ChatDatabricks(
	endpoint=endpoint_llm_a,
	max_tokens='200',
    seed=42,
    temperature=0.3
)

In [0]:
def run_llm_summary_a(llm_model=llm_model_a, df: pd.DataFrame = None):
    res = []
    for _, df_i in df.iterrows():
        res_llm = run_llm_summarization(
            llm_model=llm_model,
            input_text=df_i.article
        )
        res.append(res_llm.content)
    return res

In [0]:
run_llm_summary_a(df=df_writer_summaries.head(2))

Nick Scholfield has been booked to ride Spring Heeled in the Crabbie's Grand National at Aintree on April 11, 
replacing his original mount Sam Winner, who may not run in the £1million race.

Nick Scholfield will ride Spring Heeled in the Grand National at Aintree on April 11, replacing his original mount 
Sam Winner, who is still uncertain to run.

["Nick Scholfield has been booked to ride Spring Heeled in the Crabbie's Grand National at Aintree on April 11, replacing his original mount Sam Winner, who may not run in the £1million race.",
 'Nick Scholfield will ride Spring Heeled in the Grand National at Aintree on April 11, replacing his original mount Sam Winner, who is still uncertain to run.']

In [0]:
res_llm_a = run_llm_summary_a(df=df_writer_summaries.head(N_MAX_SAMPLES_EVAL))

Nick Scholfield is set to ride Spring Heeled in the Crabbie's Grand National at Aintree on April 11, replacing his 
original mount Sam Winner after trainer Paul Nicholls decided not to commit to running the latter.

Nick Scholfield will ride Spring Heeled in the Grand National at Aintree on April 11, replacing his original mount 
Sam Winner, who is still uncertain to run.

### 👾 `qwen3-next-80b-a3b-instruct`

In [0]:
endpoint_llm_b = SERVING_MODELS['qwen-80b']
llm_model_b = ChatDatabricks(
	endpoint=endpoint_llm_b,
	max_tokens='200',
    seed=42,
    temperature=0.2
)

In [0]:
def run_llm_summary_b(llm_model=llm_model_b, df: pd.DataFrame = None):
    res = []
    for _, df_i in df.iterrows():
        res_llm = run_llm_summarization(
            llm_model=llm_model,
            input_text=df_i.article
        )
        res.append(res_llm.content)
    return res

In [0]:
run_llm_summary_b(df=df_writer_summaries.head(2))

Nick Scholfield will ride Spring Heeled in the Grand National at Aintree after opting out of a potential ride on 
the uncertain Sam Winner, with Spring Heeled’s trainer Jim Culloty also set to run Lord Windermere with Robbie 
McNamara aboard.

Nick Scholfield will ride Spring Heeled in the Grand National at Aintree on April 11, replacing the uncertain Sam 
Winner after Paul Nicholls opted not to commit him to the race, while Robbie McNamara will ride Lord Windermere for
Jim Culloty in what will be his second National contender.

['Nick Scholfield will ride Spring Heeled in the Grand National at Aintree after opting out of a potential ride on the uncertain Sam Winner, with Spring Heeled’s trainer Jim Culloty also set to run Lord Windermere with Robbie McNamara aboard.',
 'Nick Scholfield will ride Spring Heeled in the Grand National at Aintree on April 11, replacing the uncertain Sam Winner after Paul Nicholls opted not to commit him to the race, while Robbie McNamara will ride Lord Windermere for Jim Culloty in what will be his second National contender.']

In [0]:
res_llm_b = run_llm_summary_b(df=df_writer_summaries.head(N_MAX_SAMPLES_EVAL))

Nick Scholfield will ride Spring Heeled in the Grand National at Aintree on April 11, replacing the uncertain Sam 
Winner after trainer Paul Nicholls deemed it unfair to hold him to a potential non-runner, while Robbie McNamara 
will ride Lord Windermere for Jim Culloty in the same race.

Nick Scholfield will ride Spring Heeled in the Grand National at Aintree, replacing the uncertain Sam Winner after 
Paul Nicholls deemed it unfair to hold him to a potential non-runner, while Robbie McNamara will ride Lord 
Windermere for Jim Culloty in the same race.

In [0]:
df_sample = df_writer_summaries.head(N_MAX_SAMPLES_EVAL)

In [0]:
df_sample['res_llm_a'] = res_llm_a
df_sample['res_llm_b'] = res_llm_b

### Run evaluation

In [0]:
df_sample.head(2)

,article_id,article,human_summary,res_llm_a,res_llm_b
0,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Nick Schofield is riding Spring Heeled in the ...,Nick Scholfield is set to ride Spring Heeled i...,Nick Scholfield will ride Spring Heeled in the...
1,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Preparation is taking place for a horse ridin...,Nick Scholfield will ride Spring Heeled in the...,Nick Scholfield will ride Spring Heeled in the...


In [0]:
# %pip install torch transformers textstat evaluate mlflow

In [0]:
def eval_metrics(pred_col):
    res = mlflow.evaluate(
        data=df_sample.head(N_MAX_SAMPLES_EVAL),
        targets='human_summary',
        predictions=pred_col,
        model_type='text-summarization'
    )
    return res

In [0]:
# evaluate using text-summarization model type
results_llm_a = eval_metrics('res_llm_a')
results_llm_b = eval_metrics('res_llm_b')

2026/02/05 04:02:15 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2026/02/05 04:02:17 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


In [0]:
print(f'results_llm_a:\n{results_llm_a.metrics}')
print(f'results_llm_b:\n{results_llm_b.metrics}')

results_llm_a:
{'toxicity/v1/mean': np.float64(0.00013608577864943072), 'toxicity/v1/variance': np.float64(2.705592256890217e-12),
'toxicity/v1/p90': np.float64(0.00013740167341893538), 'toxicity/v1/ratio': 0.0, 
'flesch_kincaid_grade_level/v1/mean': np.float64(14.613518518518521), 'flesch_kincaid_grade_level/v1/variance': 
np.float64(4.805026371742122), 'flesch_kincaid_grade_level/v1/p90': np.float64(16.367148148148154), 
'ari_grade_level/v1/mean': np.float64(17.39027777777778), 'ari_grade_level/v1/variance': 
np.float64(6.091298225308639), 'ari_grade_level/v1/p90': np.float64(19.364722222222223), 'rouge1/v1/mean': 
np.float64(0.3904761904761904), 'rouge1/v1/variance': np.float64(9.070294784580487e-05), 'rouge1/v1/p90': 
np.float64(0.39809523809523806), 'rouge2/v1/mean': np.float64(0.19596299411269977), 'rouge2/v1/variance': 
np.float64(0.0005545648681741309), 'rouge2/v1/p90': np.float64(0.21480235492010094), 'rougeL/v1/mean': 
np.float64(0.2833333333333333), 'rougeL/v1/variance': np.float64(0.0024999999999999988), 'rougeL/v1/p90': 
np.float64(0.3233333333333333), 'rougeLsum/v1/mean': np.float64(0.2833333333333333), 'rougeLsum/v1/variance': 
np.float64(0.0024999999999999988), 'rougeLsum/v1/p90': np.float64(0.3233333333333333)}

results_llm_b:
{'toxicity/v1/mean': np.float64(0.00019605234410846606), 'toxicity/v1/variance': 
np.float64(3.0072544909036256e-10), 'toxicity/v1/p90': np.float64(0.00020992549398215487), 'toxicity/v1/ratio': 
0.0, 'flesch_kincaid_grade_level/v1/mean': np.float64(20.564015151515154), 
'flesch_kincaid_grade_level/v1/variance': np.float64(0.5740358184113868), 'flesch_kincaid_grade_level/v1/p90': 
np.float64(21.170136363636367), 'ari_grade_level/v1/mean': np.float64(25.231505681818184), 
'ari_grade_level/v1/variance': np.float64(0.6246836591231902), 'ari_grade_level/v1/p90': 
np.float64(25.863801136363637), 'rouge1/v1/mean': np.float64(0.32291666666666663), 'rouge1/v1/variance': 
np.float64(0.00010850694444444425), 'rouge1/v1/p90': np.float64(0.33125), 'rouge2/v1/mean': 
np.float64(0.12961926091825307), 'rouge2/v1/variance': np.float64(3.840370982971957e-06), 'rouge2/v1/p90': 
np.float64(0.13118701007838746), 'rougeL/v1/mean': np.float64(0.22516025641025642), 'rougeL/v1/variance': 
np.float64(0.002086029955621305), 'rougeL/v1/p90': np.float64(0.26169871794871796), 'rougeLsum/v1/mean': 
np.float64(0.22516025641025642), 'rougeLsum/v1/variance': np.float64(0.002086029955621305), 'rougeLsum/v1/p90': 
np.float64(0.26169871794871796)}

In [0]:
df_sample

,article_id,article,human_summary,res_llm_a,res_llm_b
0,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Nick Schofield is riding Spring Heeled in the ...,Nick Scholfield is set to ride Spring Heeled i...,Nick Scholfield will ride Spring Heeled in the...
1,0adb86356834452298d180104ff54179,Nick Scholfield is lined up to ride Spring Hee...,Preparation is taking place for a horse ridin...,Nick Scholfield will ride Spring Heeled in the...,Nick Scholfield will ride Spring Heeled in the...


In [0]:
# adjust results
df_metrics_a = pd.DataFrame(results_llm_a.metrics, index=[0]).T
df_metrics_a.rename(columns={0: 'llm_a'}, inplace=True)
df_metrics_b = pd.DataFrame(results_llm_b.metrics, index=[0]).T
df_metrics_b.rename(columns={0: 'llm_b'}, inplace=True)

In [0]:
df_metrics = pd.concat([df_metrics_a, df_metrics_b], axis=1)
df_metrics

,llm_a,llm_b
toxicity/v1/mean,1.360858e-04,1.960523e-04
toxicity/v1/variance,2.705592e-12,3.007254e-10
toxicity/v1/p90,1.374017e-04,2.099255e-04
toxicity/v1/ratio,0.000000e+00,0.000000e+00
flesch_kincaid_grade_level/v1/mean,1.461352e+01,2.056402e+01
flesch_kincaid_grade_level/v1/variance,4.805026e+00,5.740358e-01
flesch_kincaid_grade_level/v1/p90,1.636715e+01,2.117014e+01
ari_grade_level/v1/mean,1.739028e+01,2.523151e+01
ari_grade_level/v1/variance,6.091298e+00,6.246837e-01
ari_grade_level/v1/p90,1.936472e+01,2.586380e+01


## Visualize results on Databricks



<img src="./imgs/evaluate_llm.png">

<img src="./imgs/results_graphs.png">